In [206]:
import numpy as np
import matplotlib.pyplot  as plt
import pandas as pd
import random

In [328]:
def sigmond(x):
    return 1/(1+np.exp(-x))

def reLu(x):
    return (np.abs(x)+x) / 2.0

def d_reLu(x):
    x[x<=0] = 0
    x[x>1] = 1
    return x

def softmax(x):
    x_row_max = x.max(axis=1).reshape(x.shape[0],1)
    x = x - x_row_max
    x_exp = np.exp(x)
    x_row_exp_sum = x_exp.sum(axis=1).reshape(x.shape[0],1)
    x_softmax = x_exp / x_row_exp_sum
    return x_softmax

def softmax_vector(x):
    x_exp = np.exp(x-x.max())
    return x_exp / x_exp.sum()

In [329]:
###### 读取数据 
stock_data = pd.read_csv('stock_data_normal_u.csv',header=None)
stock_lable = pd.read_csv('stock_data_lable_normal.txt',header=None)

In [283]:
#print(stock_lable.head(10))

In [330]:
lab_count = stock_lable.shape[0]
x = stock_data.loc[0:8999].values
print(x.shape)
x = x.reshape(-1,66)
print(x.shape)
y = stock_lable.loc[0:1499,2].values
y = y.reshape(-1,1)
print(y.shape)
#x = x / 2
x = np.insert(x,0,1,axis=1)

(9000, 11)
(1500, 66)
(1500, 1)


In [331]:
def J_softmax(h,y):
    a = 1e-10
    b = y*np.log(h+a)
    return b.sum() / 60000

In [332]:
def J_sigmond(h,y):
    J = -y*np.log(h) - (1-y)*np.log(1-h)
    return J.sum() / 1000

In [333]:
#################读取w1 与w2##################################
w1 = np.loadtxt("init_w1.txt",delimiter=',')
w2 = np.loadtxt("init_w2.txt",delimiter=',')
w2 = w2.reshape(-1,1)
#################根据文件内容，初始化w1与w2#####################

######### 设置学习率 
n = 0.8
#print(w2[0:10,:])
print(w1[0:10,0:5])
print(w2[0:10])

[[ 0.51100986  0.51848552 -0.4746211  -0.33904025  0.68943112]
 [ 1.041949   -0.4435212   0.63598022 -0.86394194  0.19830018]
 [ 0.91642987  0.12217291 -0.14872454  0.58260229  1.03943772]
 [-0.80221957 -0.6922805   0.18804048  0.17316614 -0.354452  ]
 [ 0.04028251  0.30192393  1.05466831  0.99216727 -0.67128334]
 [-0.14006993  0.06545506  0.38196629  0.04395383  0.09118204]
 [-1.02208798 -0.41597551  0.64733441  0.70925579 -0.39137275]
 [ 0.06883237  0.32369845 -0.47307993 -0.61482263 -0.05680422]
 [ 0.66318189  0.28705839  0.47475443 -0.59275355 -0.1116078 ]
 [-0.77662379 -0.34958357  0.2403412   0.52315108  0.17215894]]
[[-0.5254357 ]
 [ 0.63601096]
 [-0.46425612]
 [ 1.11622721]
 [-1.0904324 ]
 [-1.11025381]
 [ 0.38449044]
 [-1.13920051]
 [-0.00328574]
 [-0.91974108]]


In [288]:
"""
构建一个三层神经网络
一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段
取一般数据为训练样本数

数据归一化，分字段进行，然后 reshape(-1,1)
隐藏层 200 个神经元,输入到隐藏层，用reLu
输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy
w1 的 大小为 67*200
w2 的 大小为 201*1

"""

'\n构建一个三层神经网络\n一个样本为6天数据，close open high low rise exchane ma5 ma10 ma20 ma30 ma60 共11个字段，也就是 11*6 共66个字段\n取一般数据为训练样本数\n\n数据归一化，分字段进行，然后 reshape(-1,1)\n隐藏层 200 个神经元,输入到隐藏层，用reLu\n输出层就一个单元，做二分类，激活函数为 sigmond ，损失函数为 -(1-y)long(1-y) - ylogy\nw1 的 大小为 67*200\nw2 的 大小为 201*1\n\n'

In [334]:
for i in range(2000):

    ############ 隐藏层输入 ############
    hid_input = np.matmul(x,w1)  ##### 10000* 200
    ############ 隐藏层输出 ############
    hid_output = np.tanh(hid_input)
    ######## 为隐藏层增加偏执项 ############
    hid_output = np.insert(hid_output,0,1,axis=1)   # 10000* 201
    ############ 最后一层的输入 ############
    hpfs_input = np.matmul(hid_output,w2)
    ############ 最后一层的输出，即正向传播的输出值 ############
    #print(hpfs_input[0:10])
    hpfs_output = sigmond(hpfs_input)
    #print(hpfs_output[0:10])
    ############ 完成正向传播 ############

    ####################### 下面是 用reLu 和 sigmond 的情况 ###########################
    ############ w2的权重误差计算 #######################

    d_cost_hyfs_input = hpfs_output - y
    d_cost_w2 = np.matmul(hid_output.T,d_cost_hyfs_input) / 10000
    #print(d_cost_w2.shape)
    #print(d_cost_w2[100:220,:])
    ######## d_cost_w2.shape = 393*10 ############
    #print(d_cost_hyfs_input.shape,w2.shape)
    d_cost_hid_out = np.matmul(d_cost_hyfs_input,w2.T) 
    d_cost_hid_input = d_cost_hid_out*(1-hid_output*hid_output)
    d_cost_hid_input = np.delete(d_cost_hid_input,0,axis=1)
    d_cost_w1 = np.matmul(x.T,d_cost_hid_input) / 10000
    print('----------cost is -------------',J_sigmond(hpfs_output,y))
    w1 = w1 - n*d_cost_w1
    w2 = w2 - n*d_cost_w2
    #print('w1 最大值位置',np.where(w1==w1.max()),',w1 最大值',np.max(w1))
    #print('d_w1 最大值位置',np.where(d_cost_w1==d_cost_w1.max()),',d_w1 最大值',np.max(d_cost_w1))
    np.savetxt("init_w1.txt",w1,delimiter=',')
    np.savetxt("init_w2.txt",w2,delimiter=',')

print('----------------done--------------------------')

----------cost is ------------- 0.017717554561658775
----------cost is ------------- 0.017687843258119545
----------cost is ------------- 0.017660248073648622
----------cost is ------------- 0.01763405258625097
----------cost is ------------- 0.01760888248265209
----------cost is ------------- 0.017584529934112417
----------cost is ------------- 0.017560869843031797
----------cost is ------------- 0.017537819590968243
----------cost is ------------- 0.01751531946117292
----------cost is ------------- 0.017493322936981425
----------cost is ------------- 0.017471791753348256
----------cost is ------------- 0.017450693262534702
----------cost is ------------- 0.01742999894673853
----------cost is ------------- 0.01740968351482111
----------cost is ------------- 0.017389724308571417
----------cost is ------------- 0.017370100882189488
----------cost is ------------- 0.017350794685486372
----------cost is ------------- 0.01733178881397196
----------cost is ------------- 0.01731306780527064


----------cost is ------------- 0.01563225047964529
----------cost is ------------- 0.015623422843558192
----------cost is ------------- 0.015614618661688935
----------cost is ------------- 0.015605837765105832
----------cost is ------------- 0.015597079987015072
----------cost is ------------- 0.015588345162720249
----------cost is ------------- 0.015579633129582797
----------cost is ------------- 0.015570943726983522
----------cost is ------------- 0.015562276796285078
----------cost is ------------- 0.015553632180795427
----------cost is ------------- 0.015545009725732204
----------cost is ------------- 0.015536409278188027
----------cost is ------------- 0.015527830687096523
----------cost is ------------- 0.015519273803199389
----------cost is ------------- 0.015510738479014144
----------cost is ------------- 0.015502224568802694
----------cost is ------------- 0.015493731928540621
----------cost is ------------- 0.015485260415887343
----------cost is ------------- 0.0154768098901

----------cost is ------------- 0.014447405341444454
----------cost is ------------- 0.014441007304884757
----------cost is ------------- 0.01443461941853388
----------cost is ------------- 0.014428241642218037
----------cost is ------------- 0.014421873936035027
----------cost is ------------- 0.014415516260351662
----------cost is ------------- 0.014409168575801254
----------cost is ------------- 0.014402830843281122
----------cost is ------------- 0.0143965030239501
----------cost is ------------- 0.014390185079226134
----------cost is ------------- 0.01438387697078383
----------cost is ------------- 0.014377578660552124
----------cost is ------------- 0.014371290110711865
----------cost is ------------- 0.014365011283693555
----------cost is ------------- 0.014358742142175014
----------cost is ------------- 0.014352482649079108
----------cost is ------------- 0.014346232767571538
----------cost is ------------- 0.014339992461058573
----------cost is ------------- 0.0143337616931849

----------cost is ------------- 0.0135211548781817
----------cost is ------------- 0.013516014861656394
----------cost is ------------- 0.01351088082925913
----------cost is ------------- 0.013505752764834495
----------cost is ------------- 0.013500630652302672
----------cost is ------------- 0.013495514475658851
----------cost is ------------- 0.013490404218972782
----------cost is ------------- 0.013485299866388297
----------cost is ------------- 0.01348020140212278
----------cost is ------------- 0.013475108810466692
----------cost is ------------- 0.013470022075783118
----------cost is ------------- 0.013464941182507236
----------cost is ------------- 0.013459866115145932
----------cost is ------------- 0.013454796858277246
----------cost is ------------- 0.013449733396549967
----------cost is ------------- 0.013444675714683118
----------cost is ------------- 0.01343962379746556
----------cost is ------------- 0.0134345776297555
----------cost is ------------- 0.013429537196480055


----------cost is ------------- 0.012757004936916349
----------cost is ------------- 0.012752665287359131
----------cost is ------------- 0.012748329730052944
----------cost is ------------- 0.012743998256632388
----------cost is ------------- 0.012739670858761563
----------cost is ------------- 0.012735347528133914
----------cost is ------------- 0.012731028256472047
----------cost is ------------- 0.012726713035527628
----------cost is ------------- 0.012722401857081195
----------cost is ------------- 0.01271809471294205
----------cost is ------------- 0.01271379159494807
----------cost is ------------- 0.012709492494965616
----------cost is ------------- 0.012705197404889334
----------cost is ------------- 0.012700906316642061
----------cost is ------------- 0.012696619222174672
----------cost is ------------- 0.012692336113465912
----------cost is ------------- 0.012688056982522298
----------cost is ------------- 0.012683781821377938
----------cost is ------------- 0.01267951062209

KeyboardInterrupt: 

In [366]:
### 先用训练集验证

index = random.randint(0,999)
index = 707
x_train_test = x[index]
x_train_test = x_train_test.reshape(1,-1)
print(x_train_test)
print(x_train_test.shape)
print(w1.shape)
#x_train_test = np.insert(x_train_test,0,1,axis=0)
y_test = y[index]
print('input is ',index,x_train_test.shape,y_test)
hid_input = np.matmul(x_train_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
print(w2.shape)
print(hid_output.shape)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print('output is ',hpfs_output)

[[ 1.      -1.66859 -0.02712  0.31975 -1.75896  1.85628  0.43449 -1.55381
  -1.3399   0.24731 -1.65434 -1.31095  1.41188  1.76282  1.83006  1.34509
   0.17883  1.50049 -0.74312 -0.62975  0.98924 -0.67895 -0.77465  0.25671
  -0.02712 -0.0068   0.10347 -0.9177   0.60311 -0.0755  -0.32158  1.17472
   0.06694 -0.28305  0.19253 -1.32889 -0.65991  0.22763 -0.77508 -0.66662
   0.4729   0.09379 -0.30914  0.58332  0.20856 -0.32088 -0.35256 -0.65991
   0.16555 -0.06931 -0.92092  0.94977  0.72355 -0.6801   0.98495  0.74486
   0.12835 -0.02712 -0.82319 -0.08277 -0.273   -0.95055  0.94977  1.47389
  -1.42203  0.69807  1.41523]]
(1, 67)
(67, 200)
input is  707 (1, 67) [0]
(201, 1)
(1, 201)
output is  [[0.00310403]]


In [370]:
index = random.randint(0,999)
index = 707
x_test = stock_data.loc[index*6:index*6+5].values
x_test = x_test.reshape(1,-1)
x_test = np.insert(x_test,0,1,axis=1)
y_test = stock_lable.loc[index,2]
hid_input = np.matmul(x_test,w1)
hid_output = np.tanh(hid_input)
hid_output = np.insert(hid_output,0,1,axis=1)
hpfs_input = np.matmul(hid_output,w2)
hpfs_output = sigmond(hpfs_input)
print(index,x_test.shape,x_test,y_test)

707 (1, 67) [[ 1.      -1.66859 -0.02712  0.31975 -1.75896  1.85628  0.43449 -1.55381
  -1.3399   0.24731 -1.65434 -1.31095  1.41188  1.76282  1.83006  1.34509
   0.17883  1.50049 -0.74312 -0.62975  0.98924 -0.67895 -0.77465  0.25671
  -0.02712 -0.0068   0.10347 -0.9177   0.60311 -0.0755  -0.32158  1.17472
   0.06694 -0.28305  0.19253 -1.32889 -0.65991  0.22763 -0.77508 -0.66662
   0.4729   0.09379 -0.30914  0.58332  0.20856 -0.32088 -0.35256 -0.65991
   0.16555 -0.06931 -0.92092  0.94977  0.72355 -0.6801   0.98495  0.74486
   0.12835 -0.02712 -0.82319 -0.08277 -0.273   -0.95055  0.94977  1.47389
  -1.42203  0.69807  1.41523]] 0


In [ ]:
lab_count = stock_lable.shape[0]
x = stock_data.loc[0:8999].values
print(x.shape)
x = x.reshape(-1,66)
print(x.shape)
y = stock_lable.loc[0:1499,2].values
y = y.reshape(-1,1)
print(y.shape)
#x = x / 2
x = np.insert(x,0,1,axis=1)